In [2]:
import os
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from dotenv import load_dotenv
from demo_utils import demo_utils
from IPython.display import Markdown

utils = demo_utils()

In [2]:
web_source = "https://www.reuters.com/article/us-microsoft-openai-idUSKCN1UH1H9"
# web_source = "https://www.forbes.com/sites/alexandrawrage/2020/07/15/global-pharmaceutical-corruption-lessons-from-the-novartis-case/?sh=44a80a3d489b"
# web_source = "https://news.yahoo.com/roche-settles-us-patent-lawsuit-192754918.html?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuYmluZy5jb20v&guce_referrer_sig=AQAAAK6fUHnquLKmeAmg_6j8mLXq9JAsj1xlCDfKz3IgTDuPGbCND7nAVM50xH3kA4Y_l-S_PuQhY_gle227YRnvqhyaVpnSL0p_NdIpHhsk-pPhloY7x1HK_aDnAm4R35zM8anwEO9629Qv1RXfK6tva3mTfxe3HJQw2DnMtZnYelrF"

In [3]:
# web scraping
# get a web page
web_page = requests.get(web_source)
# parse the web page
web_soup = BeautifulSoup(web_page.content, 'html.parser')
# get the title
web_title = web_soup.find('title')
# get the article
web_article = web_soup.find_all('p')
# get the text
web_text = [p.get_text() for p in web_article]
plan_text = ''.join(web_text)
print(web_title, "\n",
      plan_text)

<title>Microsoft to invest $1 billion in OpenAI | Reuters</title> 
 Discover Thomson ReutersBy Stephen Nellis3 Min Read(Reuters) - Microsoft Corp said on Monday it was investing $1 billion in San Francisco-based OpenAI and that the two had formed a multi-year partnership to develop artificial intelligence supercomputing technologies on Microsoft’s Azure cloud computing service.OpenAI was founded in 2015 as a nonprofit with $1 billion in funding from Silicon Valley investors Sam Altman, Peter Thiel and LinkedIn co-founder Reid Hoffman, among others. Earlier this year, the group created a related for-profit entity to take in outside investment.Since its founding, OpenAI has employed artificial intelligence researchers to make advances in the field, such as teaching a robotic hand to perform human-like tasks entirely in software, cutting down the cost and time to train robots.The group has also focused on the safety and social implications of AI, researching how computers can generate rea

In [4]:
system_message = """
You're complinace reviewer. 
You need to collect and extract meaingful data from email, document, web page, etc.

Include following meta data for the analysis
* Source
 * Type: {Email, Document, Web page, etc.}

Extract data 
* Title
* Summary
* Organization names
* Organization industries
* People
 * Name 
 * Organization
* Events
    * Event name
    * Event date and/or time
    * Event location
"""

In [5]:
user_prompt = """
--- article ---
Title: {{title}}

{{web_data}}
------

"""

In [6]:
system_msg = {"role":"system","content":system_message}
user_msg = {"role":"user","content":user_prompt.replace("{{title}}", web_title.get_text()).replace("{{web_data}}", plan_text)}
prompt_insights = [system_msg, user_msg]
res = utils.run(prompt_insights, temperature=0.0, max_tokens=2500, top_p=0.0)
print(res)

Source
* Type: Web page

Extracted Data
* Title: Microsoft to invest $1 billion in OpenAI | Reuters
* Summary: Microsoft Corp announced its investment of $1 billion in San Francisco-based OpenAI. The two companies have formed a multi-year partnership to develop artificial intelligence supercomputing technologies on Microsoft’s Azure cloud computing service. OpenAI was founded in 2015 as a nonprofit with $1 billion in funding from Silicon Valley investors. The group has been focusing on the safety and social implications of AI. The Microsoft investment would help OpenAI pursue “artificial general intelligence,” or AGI.
* Organization names: Microsoft Corp, OpenAI, Reuters
* Organization industries: Technology, News Media
* People
    * Name: Sam Altman
    * Organization: OpenAI
    * Name: Peter Thiel
    * Organization: OpenAI
    * Name: Reid Hoffman
    * Organization: OpenAI
    * Name: Stephen Nellis
    * Organization: Reuters
* Events
    * Event name: Microsoft's investment in 

In [7]:
system_message2 = """
You're complinace reviewer. 
You need to collect and extract meaingful data from email, document, web page, etc.
Summary must be concise and clear, less than 150 words.

Include following meta data for the analysis
* Source
 * Type: {Email, Document, Web page, etc.}

Extract data 
* Title
* Summary
* Organization names
* Organization industries
* People
 * Name 
 * Organization
* Events
    * Event name
    * Event date and/or time
    * Event location

## Reponse 
Use JSON format

## Response example
```
{
    "SourceType":"WebPage",
    "Title": "Big announcement from A about B"
    "Summary":"A made an announcement about B.",
    "Organizations":[
        {
            "Name":"A",
            "Industry":"Technology"
        },
        {
            "Name":"B",
            "Industry":"Technology"
        }
    ],
    "People":[
        {
            "Name":"C",
            "Organization":"A"
        },
        {
            "Name":"D",
            "Organization":"B"
        }
    ],
    "Events":[
        {
            "Name":"Announcement",
            "Date":"2021-07-01T00:00:00Z",
            "Location":"US"
        }
    ]
}
```
"""

user_prompt2 = """
--- article ---
Title: {{title}}

{{web_data}}
------

"""

system_msg2 = {"role":"system","content":system_message2}
user_msg2 = {"role":"user","content":user_prompt2.replace("{{title}}", web_title.get_text()).replace("{{web_data}}", plan_text)}
prompt2 = [system_msg2, user_msg2]
res2 = utils.run(prompt2, temperature=0.0, max_tokens=2500, top_p=0.0)
result2 = res2.split("```")[1]
display(result2, "```")

'\n{\n    "SourceType":"WebPage",\n    "Title": "Microsoft to invest $1 billion in OpenAI | Reuters",\n    "Summary":"Microsoft Corp announced a $1 billion investment in San Francisco-based OpenAI. The two companies have formed a multi-year partnership to develop artificial intelligence supercomputing technologies on Microsoft’s Azure cloud computing service. OpenAI was founded in 2015 and has been focusing on the safety and social implications of AI. The Microsoft investment will help OpenAI pursue \'artificial general intelligence\' (AGI).",\n    "Organizations":[\n        {\n            "Name":"Microsoft Corp",\n            "Industry":"Technology"\n        },\n        {\n            "Name":"OpenAI",\n            "Industry":"Artificial Intelligence"\n        }\n    ],\n    "People":[\n        {\n            "Name":"Sam Altman",\n            "Organization":"OpenAI"\n        },\n        {\n            "Name":"Peter Thiel",\n            "Organization":"OpenAI"\n        },\n        {\n  

'```'

In [8]:
system_message3 = """
You are a compliance reviewer who monitors the news for compliance issues in an aduit team in a technology company.
The Find a Anti-Bribery and Anti-Corruption case.

## Review tasks

### 1. Explain relationship between people and organization
Understand the relationships between different people and organizations in the case.
Explain the relationship

### 2. Clssify the case
Classify the case has a potential compliance issue or not.
And explain when the event is classified.

## Reponse
Do not reveal the review task steps
Your response must be concise and clear and use Markdown format
Provide visual to show relationship between people and organization
Provide clear result at the end of the response Compliance issue or No issue with risk level, 0 is no risk, 10 is high risk

### Visual sample
Use '-, |, <, >, ...' to show relationship
A --- B --- C
      |
      |
      V
     abc, deb, gh 

### Results
Risk level: ##/10
"""

user_prompt3 = """
--- case ---
{{case_data}}
------
"""

In [9]:

system_msg3 = {"role":"system","content":system_message3}
user_msg3 = {"role":"user","content":user_prompt3.replace("{{case_data}}", result2)}
prompt3 = [system_msg3, user_msg3]
res3 = utils.run(prompt3, temperature=1.0, max_tokens=2500, top_p=0.0)
display(Markdown(res3))

### Relationship between People and Organization

In this case, the organizations involved are Microsoft Corp and OpenAI. Microsoft Corp is a technology company that announced a $1 billion investment in OpenAI, a San Francisco-based company focusing on the safety and social implications of AI. 

The people involved in this case are Sam Altman, Peter Thiel, and Reid Hoffman, all of whom are associated with OpenAI. 

The relationship can be visualized as follows:

```
Microsoft Corp --- $1 billion investment ---> OpenAI
                                               ^
                                               |
                                               |
                                               V
                                  Sam Altman, Peter Thiel, Reid Hoffman
```

### Classification of the Case

This case does not present any potential compliance issues. The event described is a legitimate business transaction where Microsoft Corp is investing in OpenAI to develop artificial intelligence supercomputing technologies. There is no mention of any illegal activities, bribery, or corruption.

### Results

Risk level: 0/10 (No risk)

In [10]:
sample_case = """
8/25/2022	Call with Nicolas Cage
9/15/2022	Call with Nicolas Cage
9/21/2022	Nicolas Cage becomes President of ABCD Cardiology Association
9/27/2022	Sponsorship payment of 1000 to ABCD Cardiology Association
10/3/2022	Call with Nicolas Cage
11/4/2022	Nicolas Cage announced as future member of ABCD Ministry of Health Formulary Committee
11/12/2022	Call with Nicolas Cage
11/20/2022	Sponsorship payment of 500 to ABCD Cardiology Association
12/16/2022	Call with Nicolas Cage
1/1/2023	Nicolas Cage effective as member of ABCD Ministry of Health Formulary Committee
1/11/2023	Call with Nicolas Cage
1/17/2023	Sponsorship payment of 500 to ABCD Cardiology Association
1/23/2023	ABCD Ministry of Health Formulary Committee approves new indication for XYZ cardiology medicine 
2/14/2023	Improper behaviour at ABCD Cardiology Association event
"""

In [11]:
system_msg4 = {"role":"system","content":system_message3}
user_msg4 = {"role":"user","content":user_prompt3.replace("{{case_data}}", sample_case)}
prompt4 = [system_msg4, user_msg4]
res4 = utils.run(prompt4, temperature=1.0, max_tokens=1000, top_p=0.0)
display(Markdown(res4))

### Relationship between People and Organization

Nicolas Cage is a key figure in this case. He is the President of the ABCD Cardiology Association and later becomes a member of the ABCD Ministry of Health Formulary Committee. The technology company in question has had multiple calls with Nicolas Cage and has made sponsorship payments to the ABCD Cardiology Association. 

The relationship can be visualized as follows:

```
Nicolas Cage --- President of ---> ABCD Cardiology Association
       |                                 |
       |                                 |
       V                                 V
Technology Company --- Sponsorship Payments ---> ABCD Cardiology Association
       |
       |
       V
Nicolas Cage --- Member of ---> ABCD Ministry of Health Formulary Committee
```

### Classification of the Case

This case has potential compliance issues. The technology company has made multiple sponsorship payments to the ABCD Cardiology Association, where Nicolas Cage is the President. Later, Nicolas Cage becomes a member of the ABCD Ministry of Health Formulary Committee, which approves a new indication for a cardiology medicine. This sequence of events raises questions about potential bribery and corruption, as the payments could be seen as an attempt to influence the decision of the Formulary Committee.

### Results

Risk level: 8/10

The risk level is high due to the potential for bribery and corruption. The timing of the sponsorship payments, the calls with Nicolas Cage, and the approval of the new indication for the cardiology medicine suggest a possible quid pro quo arrangement. This could lead to significant legal and reputational risks for the technology company.

In [12]:
# End of noteook